In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 500)

# Survey Data #

The survey responses are collected using a Google Form. The responses are automatically collected into a a Google Sheet, which is manually downloaded and placed into the raw data folder. The following cells will clean the survey data and transform it into a form easier to process and analyze.

In [2]:
survey_df = pd.read_csv("../data/raw/survey_responses.csv")
survey_df.drop("Timestamp", axis=1, inplace=True)

survey_df

,What is your participant number?,What is your major?,What is your current year?,What was your iPad number?,The number of items on the screen impacted my ability to communicate,How did the number of items on the screen affect your ability to communicate?,What do you think about the number of items on the screen?,"Overall, how much were you able to express what you wanted to communicate?",What enhanced and/or limited your ability to express your thoughts?,"I would be able to use this tool to make requests (i.e., asking for help).",Why or why not? (request),I would be able to use this tool to have a conversation,Why or why not? (converse),What is the best thing about using the app to communicate?,What is the worst thing about using the app to communicate?
0,1,Education,Third,1,4,"Some things needed to be typed, but there were...",There were too few items on the screen,I could communicate fully,The ease of usage enhanced my ability,5,"Anything that wasn't listed, could be typed so...",5,Easily used to express myself,How easy it is to use,"The automated voice, could be humanized a litt..."
1,2,Computer Science,Fifth or more,2,5,I had trouble finding the basic structural wor...,There were too many items on the screen,I could communicate most of my thoughts,A lot of regularly used words were on the main...,3,When asking for help I would only respond vagu...,3,I could use the tool but it would take way too...,When the category tabs had words inside of the...,There are too many images on the screen at one...
2,3,Education,Fourth,3,5,There were too many options. I would give up a...,There were too many items on the screen,I could communicate some of my thoughts,Not being able to find what I wanted fast enou...,4,Help was easy to find,2,It takes a long time and people feel impatient,not have to use voice to communicate,hard to find options
3,4,Education,Third,1,1,The number of items on the screen only affecte...,There were too few items on the screen,I could communicate fully,My preconceptions of the app. I thought I was ...,5,I would be able to use this tool to make a req...,5,I would be able to use this tool to have a con...,The best thing about using the app to communic...,The worst thing about using the app is that I ...
4,5,Education,Fourth,2,5,The number of items affected my ability becaus...,There were too few items on the screen,I could communicate most of my thoughts,To me I wanted to make a full sentence so that...,5,I feel that asking help in general is a common...,4,If you knew in an advance what your conversati...,The pictures help give you an idea what it is ...,It takes an extremely long time trying to get ...
5,6,Computer Science,Fourth,3,5,"To many options on screen, I felt overwhelmed ...",There were too many items on the screen,I could communicate some of my thoughts,What limited my thoughts was just knowledge of...,2,Requests such as going to the bathroom and ask...,1,Conversations are extremely fragmented. It is ...,It is better then using nothing or attempting ...,The entire thing feels almost unusable. Nothin...
6,7,Education,Third,1,3,There were not enough options toward school re...,There were too few items on the screen,I could communicate fully,The fact that when I was typing it said the le...,5,Because there was a help question and the key ...,4,"Yes, but there is a lot of typing that is need...",I think the folders was a really great idea to...,"There was too much typing, I think it needs mo..."
7,8,Computer Science,Fifth or more,2,4,The number of items on any one screen felt app...,The number of items on the screen was appropriate,I could communicate most of my thoughts,The lack of options as I drilled down into men...,3,I could certainly ask for help with simple tas...,3,"Yes, I'd be able to have a basic conversation,...",For me it took a lot longer than just typing a...,Not finding words that'd you'd expect to find ...
8,9,Computer Science,Second,3,4,It negatively affected my ability to communica...,There

Since the labels in the spreadsheet are the questions in the form, they are changed to something easier to use in code. There are two columns with the text "Why or why not?". These are manually updated in the raw data file to add extra text, either "(request)" or "(converse)", to make them two distinct columns.

In [3]:
labels = {"What is your participant number?":"p_number",
          "What is your major?":"major",
          "What is your current year?":"year",
          "What was your iPad number?":"screen_size",
          "The number of items on the screen impacted my ability to communicate":"num_items_impact_scale",
          "How did the number of items on the screen affect your ability to communicate?":"num_items_impact_text",
          "What do you think about the number of items on the screen?":"num_items_thoughts_scale",
          "Overall, how much were you able to express what you wanted to communicate?":"expression_scale",
          "What enhanced and/or limited your ability to express your thoughts?":"expression_text",
          "I would be able to use this tool to make requests (i.e., asking for help).":"request_scale",
          "Why or why not? (request)":"request_rationale",
          "I would be able to use this tool to have a conversation":"converse_scale",
          "Why or why not? (converse)":"consverse_rationale",
          "What is the best thing about using the app to communicate?":"best_text",
          "What is the worst thing about using the app to communicate?":"worst_text"}
survey_df = survey_df.rename(columns=labels)
survey_df

,p_number,major,year,screen_size,num_items_impact_scale,num_items_impact_text,num_items_thoughts_scale,expression_scale,expression_text,request_scale,request_rationale,converse_scale,consverse_rationale,best_text,worst_text
0,1,Education,Third,1,4,"Some things needed to be typed, but there were...",There were too few items on the screen,I could communicate fully,The ease of usage enhanced my ability,5,"Anything that wasn't listed, could be typed so...",5,Easily used to express myself,How easy it is to use,"The automated voice, could be humanized a litt..."
1,2,Computer Science,Fifth or more,2,5,I had trouble finding the basic structural wor...,There were too many items on the screen,I could communicate most of my thoughts,A lot of regularly used words were on the main...,3,When asking for help I would only respond vagu...,3,I could use the tool but it would take way too...,When the category tabs had words inside of the...,There are too many images on the screen at one...
2,3,Education,Fourth,3,5,There were too many options. I would give up a...,There were too many items on the screen,I could communicate some of my thoughts,Not being able to find what I wanted fast enou...,4,Help was easy to find,2,It takes a long time and people feel impatient,not have to use voice to communicate,hard to find options
3,4,Education,Third,1,1,The number of items on the screen only affecte...,There were too few items on the screen,I could communicate fully,My preconceptions of the app. I thought I was ...,5,I would be able to use this tool to make a req...,5,I would be able to use this tool to have a con...,The best thing about using the app to communic...,The worst thing about using the app is that I ...
4,5,Education,Fourth,2,5,The number of items affected my ability becaus...,There were too few items on the screen,I could communicate most of my thoughts,To me I wanted to make a full sentence so that...,5,I feel that asking help in general is a common...,4,If you knew in an advance what your conversati...,The pictures help give you an idea what it is ...,It takes an extremely long time trying to get ...
5,6,Computer Science,Fourth,3,5,"To many options on screen, I felt overwhelmed ...",There were too many items on the screen,I could communicate some of my thoughts,What limited my thoughts was just knowledge of...,2,Requests such as going to the bathroom and ask...,1,Conversations are extremely fragmented. It is ...,It is better then using nothing or attempting ...,The entire thing feels almost unusable. Nothin...
6,7,Education,Third,1,3,There were not enough options toward school re...,There were too few items on the screen,I could communicate fully,The fact that when I was typing it said the le...,5,Because there was a help question and the key ...,4,"Yes, but there is a lot of typing that is need...",I think the folders was a really great idea to...,"There was too much typing, I think it needs mo..."
7,8,Computer Science,Fifth or more,2,4,The number of items on any one screen felt app...,The number of items on the screen was appropriate,I could communicate most of my thoughts,The lack of options as I drilled down into men...,3,I could certainly ask for help with simple tas...,3,"Yes, I'd be able to have a basic conversation,...",For me it took a lot longer than just typing a...,Not finding words that'd you'd expect to find ...
8,9,Computer Science,Second,3,4,It negatively affected my ability to communica...,There were too many items on the screen,I could communicate some of my thoughts,"The color grouping was helpful, but sometimes ...",4,The help and question icons were clearly ident...,3,Some extraneous icons were either difficult to...,The color grouping on the main page.,Some of the organization of the quarter circle...
9,10,Education,Third,1,4,At times it felt like there were too many item...,The number of items on the screen was appropriate,I could communicate most of my thoughts,Having the keyboard really enhanced my ability...,4,Yes f

This cell will parse the year from a string into integers. Years "First" through "Fourth" correspond to the integers 1-4, with 5 being chosen to represent the string "Fifth or more".

In [4]:
def parse_year(s):
    if s == "First":
        return 1
    elif s == "Second":
        return 2
    elif s == "Third":
        return 3
    elif s == "Fourth":
        return 4
    else:
        return 5
    
survey_df["year"] = survey_df["year"].apply(parse_year)
survey_df

,p_number,major,year,screen_size,num_items_impact_scale,num_items_impact_text,num_items_thoughts_scale,expression_scale,expression_text,request_scale,request_rationale,converse_scale,consverse_rationale,best_text,worst_text
0,1,Education,3,1,4,"Some things needed to be typed, but there were...",There were too few items on the screen,I could communicate fully,The ease of usage enhanced my ability,5,"Anything that wasn't listed, could be typed so...",5,Easily used to express myself,How easy it is to use,"The automated voice, could be humanized a litt..."
1,2,Computer Science,5,2,5,I had trouble finding the basic structural wor...,There were too many items on the screen,I could communicate most of my thoughts,A lot of regularly used words were on the main...,3,When asking for help I would only respond vagu...,3,I could use the tool but it would take way too...,When the category tabs had words inside of the...,There are too many images on the screen at one...
2,3,Education,4,3,5,There were too many options. I would give up a...,There were too many items on the screen,I could communicate some of my thoughts,Not being able to find what I wanted fast enou...,4,Help was easy to find,2,It takes a long time and people feel impatient,not have to use voice to communicate,hard to find options
3,4,Education,3,1,1,The number of items on the screen only affecte...,There were too few items on the screen,I could communicate fully,My preconceptions of the app. I thought I was ...,5,I would be able to use this tool to make a req...,5,I would be able to use this tool to have a con...,The best thing about using the app to communic...,The worst thing about using the app is that I ...
4,5,Education,4,2,5,The number of items affected my ability becaus...,There were too few items on the screen,I could communicate most of my thoughts,To me I wanted to make a full sentence so that...,5,I feel that asking help in general is a common...,4,If you knew in an advance what your conversati...,The pictures help give you an idea what it is ...,It takes an extremely long time trying to get ...
5,6,Computer Science,4,3,5,"To many options on screen, I felt overwhelmed ...",There were too many items on the screen,I could communicate some of my thoughts,What limited my thoughts was just knowledge of...,2,Requests such as going to the bathroom and ask...,1,Conversations are extremely fragmented. It is ...,It is better then using nothing or attempting ...,The entire thing feels almost unusable. Nothin...
6,7,Education,3,1,3,There were not enough options toward school re...,There were too few items on the screen,I could communicate fully,The fact that when I was typing it said the le...,5,Because there was a help question and the key ...,4,"Yes, but there is a lot of typing that is need...",I think the folders was a really great idea to...,"There was too much typing, I think it needs mo..."
7,8,Computer Science,5,2,4,The number of items on any one screen felt app...,The number of items on the screen was appropriate,I could communicate most of my thoughts,The lack of options as I drilled down into men...,3,I could certainly ask for help with simple tas...,3,"Yes, I'd be able to have a basic conversation,...",For me it took a lot longer than just typing a...,Not finding words that'd you'd expect to find ...
8,9,Computer Science,2,3,4,It negatively affected my ability to communica...,There were too many items on the screen,I could communicate some of my thoughts,"The color grouping was helpful, but sometimes ...",4,The help and question icons were clearly ident...,3,Some extraneous icons were either difficult to...,The color grouping on the main page.,Some of the organization of the quarter circle...
9,10,Education,3,1,4,At times it felt like there were too many item...,The number of items on the screen was appropriate,I could communicate most of my thoughts,Having the keyboard really enhanced my ability...,4,Yes for the most part I could use this tool to...,3,"I could have

This cell will turn the iPad number into a categorical variable for screen size.

In [5]:
def parse_size(s):
    if s == 1:
        return "Small"
    elif s == 2:
        return "Medium"
    else:
        return "Large"
    
survey_df["screen_size"] = survey_df["screen_size"].apply(parse_size)

This cell will parse the data for the question "What do you think about the number of items on the screen?", which had three options: too few, too many, just right. If the respondent chose too few, -1 is return. Too many, 1. Just right, 0.

In [6]:
def parse_scale(s):
    if s == "There were too few items on the screen":
        return 1
    elif s == "There were too many items on the screen":
        return 2
    else:
        return 3

survey_df["num_items_thoughts_scale"] = survey_df["num_items_thoughts_scale"].apply(parse_scale)
survey_df

,p_number,major,year,screen_size,num_items_impact_scale,num_items_impact_text,num_items_thoughts_scale,expression_scale,expression_text,request_scale,request_rationale,converse_scale,consverse_rationale,best_text,worst_text
0,1,Education,3,Small,4,"Some things needed to be typed, but there were...",1,I could communicate fully,The ease of usage enhanced my ability,5,"Anything that wasn't listed, could be typed so...",5,Easily used to express myself,How easy it is to use,"The automated voice, could be humanized a litt..."
1,2,Computer Science,5,Medium,5,I had trouble finding the basic structural wor...,2,I could communicate most of my thoughts,A lot of regularly used words were on the main...,3,When asking for help I would only respond vagu...,3,I could use the tool but it would take way too...,When the category tabs had words inside of the...,There are too many images on the screen at one...
2,3,Education,4,Large,5,There were too many options. I would give up a...,2,I could communicate some of my thoughts,Not being able to find what I wanted fast enou...,4,Help was easy to find,2,It takes a long time and people feel impatient,not have to use voice to communicate,hard to find options
3,4,Education,3,Small,1,The number of items on the screen only affecte...,1,I could communicate fully,My preconceptions of the app. I thought I was ...,5,I would be able to use this tool to make a req...,5,I would be able to use this tool to have a con...,The best thing about using the app to communic...,The worst thing about using the app is that I ...
4,5,Education,4,Medium,5,The number of items affected my ability becaus...,1,I could communicate most of my thoughts,To me I wanted to make a full sentence so that...,5,I feel that asking help in general is a common...,4,If you knew in an advance what your conversati...,The pictures help give you an idea what it is ...,It takes an extremely long time trying to get ...
5,6,Computer Science,4,Large,5,"To many options on screen, I felt overwhelmed ...",2,I could communicate some of my thoughts,What limited my thoughts was just knowledge of...,2,Requests such as going to the bathroom and ask...,1,Conversations are extremely fragmented. It is ...,It is better then using nothing or attempting ...,The entire thing feels almost unusable. Nothin...
6,7,Education,3,Small,3,There were not enough options toward school re...,1,I could communicate fully,The fact that when I was typing it said the le...,5,Because there was a help question and the key ...,4,"Yes, but there is a lot of typing that is need...",I think the folders was a really great idea to...,"There was too much typing, I think it needs mo..."
7,8,Computer Science,5,Medium,4,The number of items on any one screen felt app...,3,I could communicate most of my thoughts,The lack of options as I drilled down into men...,3,I could certainly ask for help with simple tas...,3,"Yes, I'd be able to have a basic conversation,...",For me it took a lot longer than just typing a...,Not finding words that'd you'd expect to find ...
8,9,Computer Science,2,Large,4,It negatively affected my ability to communica...,2,I could communicate some of my thoughts,"The color grouping was helpful, but sometimes ...",4,The help and question icons were clearly ident...,3,Some extraneous icons were either difficult to...,The color grouping on the main page.,Some of the organization of the quarter circle...
9,10,Education,3,Small,4,At times it felt like there were too many item...,3,I could communicate most of my thoughts,Having the keyboard really enhanced my ability...,4,Yes for the most part I could use this tool to...,3,"I could have limited conversation, and it woul...",The best thing about using this app is being a...,It was frustrating when I was looking for a wo...


This cell parses the scale used to measure the participants thoughts on their ability to communicate. A four-point scale is used, ranging from "I could not communicate any thoughts" to "I could communicate fully". The integer scale chosen to represent the strings is 0-3, with 0 corresponding to "I could not communicate any thoughts" and 3 to "I could communicate fully".

In [7]:
def parse_scale(s):
    if s == "I could not communicate any thoughts":
        return 1
    elif s == "I could communicate some of my thoughts":
        return 2
    elif s == "I could communicate most of my thoughts":
        return 3
    else:
        return 4
    
survey_df["expression_scale"] = survey_df["expression_scale"].apply(parse_scale)
survey_df

,p_number,major,year,screen_size,num_items_impact_scale,num_items_impact_text,num_items_thoughts_scale,expression_scale,expression_text,request_scale,request_rationale,converse_scale,consverse_rationale,best_text,worst_text
0,1,Education,3,Small,4,"Some things needed to be typed, but there were...",1,4,The ease of usage enhanced my ability,5,"Anything that wasn't listed, could be typed so...",5,Easily used to express myself,How easy it is to use,"The automated voice, could be humanized a litt..."
1,2,Computer Science,5,Medium,5,I had trouble finding the basic structural wor...,2,3,A lot of regularly used words were on the main...,3,When asking for help I would only respond vagu...,3,I could use the tool but it would take way too...,When the category tabs had words inside of the...,There are too many images on the screen at one...
2,3,Education,4,Large,5,There were too many options. I would give up a...,2,2,Not being able to find what I wanted fast enou...,4,Help was easy to find,2,It takes a long time and people feel impatient,not have to use voice to communicate,hard to find options
3,4,Education,3,Small,1,The number of items on the screen only affecte...,1,4,My preconceptions of the app. I thought I was ...,5,I would be able to use this tool to make a req...,5,I would be able to use this tool to have a con...,The best thing about using the app to communic...,The worst thing about using the app is that I ...
4,5,Education,4,Medium,5,The number of items affected my ability becaus...,1,3,To me I wanted to make a full sentence so that...,5,I feel that asking help in general is a common...,4,If you knew in an advance what your conversati...,The pictures help give you an idea what it is ...,It takes an extremely long time trying to get ...
5,6,Computer Science,4,Large,5,"To many options on screen, I felt overwhelmed ...",2,2,What limited my thoughts was just knowledge of...,2,Requests such as going to the bathroom and ask...,1,Conversations are extremely fragmented. It is ...,It is better then using nothing or attempting ...,The entire thing feels almost unusable. Nothin...
6,7,Education,3,Small,3,There were not enough options toward school re...,1,4,The fact that when I was typing it said the le...,5,Because there was a help question and the key ...,4,"Yes, but there is a lot of typing that is need...",I think the folders was a really great idea to...,"There was too much typing, I think it needs mo..."
7,8,Computer Science,5,Medium,4,The number of items on any one screen felt app...,3,3,The lack of options as I drilled down into men...,3,I could certainly ask for help with simple tas...,3,"Yes, I'd be able to have a basic conversation,...",For me it took a lot longer than just typing a...,Not finding words that'd you'd expect to find ...
8,9,Computer Science,2,Large,4,It negatively affected my ability to communica...,2,2,"The color grouping was helpful, but sometimes ...",4,The help and question icons were clearly ident...,3,Some extraneous icons were either difficult to...,The color grouping on the main page.,Some of the organization of the quarter circle...
9,10,Education,3,Small,4,At times it felt like there were too many item...,3,3,Having the keyboard really enhanced my ability...,4,Yes for the most part I could use this tool to...,3,"I could have limited conversation, and it woul...",The best thing about using this app is being a...,It was frustrating when I was looking for a wo...


The data is then written to the processes CSV.

In [8]:
survey_df.to_csv('../data/processed/survey_data.csv', index=False)
survey_df

,p_number,major,year,screen_size,num_items_impact_scale,num_items_impact_text,num_items_thoughts_scale,expression_scale,expression_text,request_scale,request_rationale,converse_scale,consverse_rationale,best_text,worst_text
0,1,Education,3,Small,4,"Some things needed to be typed, but there were...",1,4,The ease of usage enhanced my ability,5,"Anything that wasn't listed, could be typed so...",5,Easily used to express myself,How easy it is to use,"The automated voice, could be humanized a litt..."
1,2,Computer Science,5,Medium,5,I had trouble finding the basic structural wor...,2,3,A lot of regularly used words were on the main...,3,When asking for help I would only respond vagu...,3,I could use the tool but it would take way too...,When the category tabs had words inside of the...,There are too many images on the screen at one...
2,3,Education,4,Large,5,There were too many options. I would give up a...,2,2,Not being able to find what I wanted fast enou...,4,Help was easy to find,2,It takes a long time and people feel impatient,not have to use voice to communicate,hard to find options
3,4,Education,3,Small,1,The number of items on the screen only affecte...,1,4,My preconceptions of the app. I thought I was ...,5,I would be able to use this tool to make a req...,5,I would be able to use this tool to have a con...,The best thing about using the app to communic...,The worst thing about using the app is that I ...
4,5,Education,4,Medium,5,The number of items affected my ability becaus...,1,3,To me I wanted to make a full sentence so that...,5,I feel that asking help in general is a common...,4,If you knew in an advance what your conversati...,The pictures help give you an idea what it is ...,It takes an extremely long time trying to get ...
5,6,Computer Science,4,Large,5,"To many options on screen, I felt overwhelmed ...",2,2,What limited my thoughts was just knowledge of...,2,Requests such as going to the bathroom and ask...,1,Conversations are extremely fragmented. It is ...,It is better then using nothing or attempting ...,The entire thing feels almost unusable. Nothin...
6,7,Education,3,Small,3,There were not enough options toward school re...,1,4,The fact that when I was typing it said the le...,5,Because there was a help question and the key ...,4,"Yes, but there is a lot of typing that is need...",I think the folders was a really great idea to...,"There was too much typing, I think it needs mo..."
7,8,Computer Science,5,Medium,4,The number of items on any one screen felt app...,3,3,The lack of options as I drilled down into men...,3,I could certainly ask for help with simple tas...,3,"Yes, I'd be able to have a basic conversation,...",For me it took a lot longer than just typing a...,Not finding words that'd you'd expect to find ...
8,9,Computer Science,2,Large,4,It negatively affected my ability to communica...,2,2,"The color grouping was helpful, but sometimes ...",4,The help and question icons were clearly ident...,3,Some extraneous icons were either difficult to...,The color grouping on the main page.,Some of the organization of the quarter circle...
9,10,Education,3,Small,4,At times it felt like there were too many item...,3,3,Having the keyboard really enhanced my ability...,4,Yes for the most part I could use this tool to...,3,"I could have limited conversation, and it woul...",The best thing about using this app is being a...,It was frustrating when I was looking for a wo...


# Task Data #

The task data was manually collected during the participation process. The data was collected into a Google Sheet with the following encoding scheme:

```
Answer Type:
     S : single word
     F : sentence fragment
     C : complete sentence
     I : incomplete; the participate chose to not finish the task

Success:
     + : the task was successfully completed
     - : the task was not successfully completed

Length:
     Integer : how many words were in the responses to the task
```

For example, a response of 4 words that made a sentence fragment which successfully completed the task would be encoded as F+4.

In [9]:
task_df = pd.read_csv('../data/raw/task_data.csv')

task_df

,Participant Number,iPad Number,T1 Snack,T2 Bathroom,T3 Marker,T4 Multiply,T5 Use Tool,T6 Movie,T7 Breadstick,T8 Nickname,T9 Joke,T10 Weekend
0,1,1,C+6,F+4,F+4,C+5,C+3,C+6,C+4,C+2,C+11,F+2
1,2,2,C+7,C+7,C+5,C+6,C+9,C+6,C+5,C+3,C-2,C+7
2,3,3,F+3,F+4,F+4,C+7,F+2,F+4,C+3,C+2,F+8,C+3
3,4,1,C+3,C+6,C+5,C+6,C+3,S-1,S+1,F+2,I,C+3
4,5,2,C+4,F+4,F+5,C+6,C+6,C+6,S+1,C+4,C-5,F+3
5,6,3,F+6,C+5,F+4,F+5,F+6,F+6,F+6,C+2,C+2,F+8
6,7,1,F+3,F+4,C+5,C+6,C+7,C+6,C+6,C+2,C+13,F+5
7,8,2,F+4,F+3,F+2,F+8,C+7,F+5,F+5,C+2,F+11,F+6
8,9,3,C+6,C+6,C+4,F+5,C+4,C+7,C+4,C+5,C+11,C+6
9,10,1,F+4,F+3,F+4,F+5,F+4,C+7,F+4,C+2,F+7,C+6


The labels in the CSV are the names given to each column in the Google Sheet. They are adjusted here to be easier to use in code.

In [10]:
labels = {"Participant Number":"p_number",
          "iPad Number":"screen_size",
          "T1 Snack":"t1",
          "T2 Bathroom":"t2",
          "T3 Marker":"t3",
          "T4 Multiply":"t4",
          "T5 Use Tool":"t5",
          "T6 Movie":"t6",
          "T7 Breadstick":"t7",
          "T8 Nickname":"t8",
          "T9 Joke":"t9",
          "T10 Weekend":"t10"}
task_df = task_df.rename(columns=labels)
task_df

,p_number,screen_size,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
0,1,1,C+6,F+4,F+4,C+5,C+3,C+6,C+4,C+2,C+11,F+2
1,2,2,C+7,C+7,C+5,C+6,C+9,C+6,C+5,C+3,C-2,C+7
2,3,3,F+3,F+4,F+4,C+7,F+2,F+4,C+3,C+2,F+8,C+3
3,4,1,C+3,C+6,C+5,C+6,C+3,S-1,S+1,F+2,I,C+3
4,5,2,C+4,F+4,F+5,C+6,C+6,C+6,S+1,C+4,C-5,F+3
5,6,3,F+6,C+5,F+4,F+5,F+6,F+6,F+6,C+2,C+2,F+8
6,7,1,F+3,F+4,C+5,C+6,C+7,C+6,C+6,C+2,C+13,F+5
7,8,2,F+4,F+3,F+2,F+8,C+7,F+5,F+5,C+2,F+11,F+6
8,9,3,C+6,C+6,C+4,F+5,C+4,C+7,C+4,C+5,C+11,C+6
9,10,1,F+4,F+3,F+4,F+5,F+4,C+7,F+4,C+2,F+7,C+6


This cell adds the major to the task data

In [11]:
temp_df = survey_df[["p_number", "major"]]
task_df = pd.merge(left=temp_df, right=task_df, on="p_number")
task_df

,p_number,major,screen_size,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
0,1,Education,1,C+6,F+4,F+4,C+5,C+3,C+6,C+4,C+2,C+11,F+2
1,2,Computer Science,2,C+7,C+7,C+5,C+6,C+9,C+6,C+5,C+3,C-2,C+7
2,3,Education,3,F+3,F+4,F+4,C+7,F+2,F+4,C+3,C+2,F+8,C+3
3,4,Education,1,C+3,C+6,C+5,C+6,C+3,S-1,S+1,F+2,I,C+3
4,5,Education,2,C+4,F+4,F+5,C+6,C+6,C+6,S+1,C+4,C-5,F+3
5,6,Computer Science,3,F+6,C+5,F+4,F+5,F+6,F+6,F+6,C+2,C+2,F+8
6,7,Education,1,F+3,F+4,C+5,C+6,C+7,C+6,C+6,C+2,C+13,F+5
7,8,Computer Science,2,F+4,F+3,F+2,F+8,C+7,F+5,F+5,C+2,F+11,F+6
8,9,Computer Science,3,C+6,C+6,C+4,F+5,C+4,C+7,C+4,C+5,C+11,C+6
9,10,Education,1,F+4,F+3,F+4,F+5,F+4,C+7,F+4,C+2,F+7,C+6


This cell changed the iPad number to categorical variable, the same as above with the survey data.

In [12]:
def parse_size(s):
    if s == 1:
        return "Small"
    elif s == 2:
        return "Medium"
    else:
        return "Large"
    
task_df["screen_size"] = task_df["screen_size"].apply(parse_size)

The following function will parse out my encoding scheme into the three values. These values are then transformed into the appropriate data types and stored as separate columns in the clean data.

In [13]:
def print_vals(df):
    resultsDf = pd.DataFrame()
    for label, content in df.items():
        types = []
        success = []
        length = []
        for index, value in content.items():
            if len(value) > 1:
                ans_type = value[0]
                is_success = value[1]
                ans_length = int(value[2:])
            else:
                ans_type = "I"
                is_success = False
                ans_length = 0

#             if ans_type == "I":
#                 ans_type = 0
#             elif ans_type == "S":
#                 ans_type = 1
#             elif ans_type == "F":
#                 ans_type = 2
#             elif ans_type == "C":
#                 ans_type = 3

            if is_success == "+":
                is_success = True
            else:
                is_success = False
                
            types.append(ans_type)
            success.append(is_success)
            length.append(ans_length)
            
        types_label = label + "_type"
        success_label = label + "_isSuccess"
        length_label = label + "_numWords"
        
        
        resultsDf[types_label] = types
        resultsDf[success_label] = success
        resultsDf[length_label] = length
        
    return resultsDf

The task columns are parsed and stored in a temporary DataFrame. This temporary DataFrame is then concatenated with the participant number and iPad number columns to create a complete, clean DataFrame.

In [14]:
new_df = print_vals(task_df[["t1","t2","t3","t4","t5","t6","t7","t8","t9","t10"]])
frames = [task_df[["p_number", "major", "screen_size"]], new_df]
clean_df = pd.concat(frames, axis=1)

Unfortunately, not all participants completed the post survey. The next two cells find the participants numbers of those who completed the survey and drops those rows from the task data DataFrame.

In [15]:
participants = survey_df["p_number"].to_list()
participants

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 18, 19, 20]

In [16]:
clean_df = clean_df[clean_df["p_number"].isin(participants)]
clean_df

,p_number,major,screen_size,t1_type,t1_isSuccess,t1_numWords,t2_type,t2_isSuccess,t2_numWords,t3_type,t3_isSuccess,t3_numWords,t4_type,t4_isSuccess,t4_numWords,t5_type,t5_isSuccess,t5_numWords,t6_type,t6_isSuccess,t6_numWords,t7_type,t7_isSuccess,t7_numWords,t8_type,t8_isSuccess,t8_numWords,t9_type,t9_isSuccess,t9_numWords,t10_type,t10_isSuccess,t10_numWords
0,1,Education,Small,C,True,6,F,True,4,F,True,4,C,True,5,C,True,3,C,True,6,C,True,4,C,True,2,C,True,11,F,True,2
1,2,Computer Science,Medium,C,True,7,C,True,7,C,True,5,C,True,6,C,True,9,C,True,6,C,True,5,C,True,3,C,False,2,C,True,7
2,3,Education,Large,F,True,3,F,True,4,F,True,4,C,True,7,F,True,2,F,True,4,C,True,3,C,True,2,F,True,8,C,True,3
3,4,Education,Small,C,True,3,C,True,6,C,True,5,C,True,6,C,True,3,S,False,1,S,True,1,F,True,2,I,False,0,C,True,3
4,5,Education,Medium,C,True,4,F,True,4,F,True,5,C,True,6,C,True,6,C,True,6,S,True,1,C,True,4,C,False,5,F,True,3
5,6,Computer Science,Large,F,True,6,C,True,5,F,True,4,F,True,5,F,True,6,F,True,6,F,True,6,C,True,2,C,True,2,F,True,8
6,7,Education,Small,F,True,3,F,True,4,C,True,5,C,True,6,C,True,7,C,True,6,C,True,6,C,True,2,C,True,13,F,True,5
7,8,Computer Science,Medium,F,True,4,F,True,3,F,True,2,F,True,8,C,True,7,F,True,5,F,True,5,C,True,2,F,True,11,F,True,6
8,9,Computer Science,Large,C,True,6,C,True,6,C,True,4,F,True,5,C,True,4,C,True,7,C,True,4,C,True,5,C,True,11,C,True,6
9,10,Education,Small,F,True,4,F,True,3,F,True,4,F,True,5,F,True,4,C,True,7,F,True,4,C,True,2,F,True,7,C,True,6


The subset of task data is then written to the processed data CSV.

In [17]:
clean_df.to_csv('../data/processed/task_data.csv', index=False)
clean_df

,p_number,major,screen_size,t1_type,t1_isSuccess,t1_numWords,t2_type,t2_isSuccess,t2_numWords,t3_type,t3_isSuccess,t3_numWords,t4_type,t4_isSuccess,t4_numWords,t5_type,t5_isSuccess,t5_numWords,t6_type,t6_isSuccess,t6_numWords,t7_type,t7_isSuccess,t7_numWords,t8_type,t8_isSuccess,t8_numWords,t9_type,t9_isSuccess,t9_numWords,t10_type,t10_isSuccess,t10_numWords
0,1,Education,Small,C,True,6,F,True,4,F,True,4,C,True,5,C,True,3,C,True,6,C,True,4,C,True,2,C,True,11,F,True,2
1,2,Computer Science,Medium,C,True,7,C,True,7,C,True,5,C,True,6,C,True,9,C,True,6,C,True,5,C,True,3,C,False,2,C,True,7
2,3,Education,Large,F,True,3,F,True,4,F,True,4,C,True,7,F,True,2,F,True,4,C,True,3,C,True,2,F,True,8,C,True,3
3,4,Education,Small,C,True,3,C,True,6,C,True,5,C,True,6,C,True,3,S,False,1,S,True,1,F,True,2,I,False,0,C,True,3
4,5,Education,Medium,C,True,4,F,True,4,F,True,5,C,True,6,C,True,6,C,True,6,S,True,1,C,True,4,C,False,5,F,True,3
5,6,Computer Science,Large,F,True,6,C,True,5,F,True,4,F,True,5,F,True,6,F,True,6,F,True,6,C,True,2,C,True,2,F,True,8
6,7,Education,Small,F,True,3,F,True,4,C,True,5,C,True,6,C,True,7,C,True,6,C,True,6,C,True,2,C,True,13,F,True,5
7,8,Computer Science,Medium,F,True,4,F,True,3,F,True,2,F,True,8,C,True,7,F,True,5,F,True,5,C,True,2,F,True,11,F,True,6
8,9,Computer Science,Large,C,True,6,C,True,6,C,True,4,F,True,5,C,True,4,C,True,7,C,True,4,C,True,5,C,True,11,C,True,6
9,10,Education,Small,F,True,4,F,True,3,F,True,4,F,True,5,F,True,4,C,True,7,F,True,4,C,True,2,F,True,7,C,True,6
